In [35]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

In [2]:
train_meta_path = './kaggle/input/icecube-neutrinos-in-deep-ice/train_meta.parquet'
train_path_list = [
    './kaggle/input/icecube-neutrinos-in-deep-ice/batch_1.parquet'
]
sensor_geometry_path = './kaggle/input/icecube-neutrinos-in-deep-ice/sensor_geometry.csv'

In [3]:
train_meta = pd.read_parquet(train_meta_path)
train_meta

,batch_id,event_id,first_pulse_index,last_pulse_index,azimuth,zenith
0,1,24,0,60,5.029555,2.087498
1,1,41,61,111,0.417742,1.549686
2,1,59,112,147,1.160466,2.401942
3,1,67,148,289,5.845952,0.759054
4,1,72,290,351,0.653719,0.939117
...,...,...,...,...,...,...
199995,1,3266160,32791961,32792033,1.536491,0.537451
199996,1,3266175,32792034,32792157,5.091808,2.732550
199997,1,3266192,32792158,32792216,3.605875,0.815684
199998,1,3266195,32792217,32792279,3.752771,1.098687


In [4]:
train = pd.read_parquet(train_path_list).reset_index()

In [5]:
train.loc[train['event_id'] == 24]

,event_id,sensor_id,time,charge,auxiliary
0,24,3918,5928,1.325,True
1,24,4157,6115,1.175,True
2,24,3520,6492,0.925,True
3,24,5041,6665,0.225,True
4,24,2948,8054,1.575,True
...,...,...,...,...,...
56,24,3452,17812,0.725,True
57,24,48,18053,0.975,True
58,24,3267,18095,1.125,True
59,24,3267,18102,1.425,True


In [6]:
sensor_geometry = pd.read_csv(sensor_geometry_path)

In [84]:
sensor_fig = px.scatter_3d(sensor_geometry, x="x", y="y", z="z", opacity=0.3)
sensor_fig.update_traces(marker_size=0.5)
sensor_fig

In [104]:
## Each train event is a sensor flaring up.
## We want a dataframe showing the rolling max of each sensor over the entire collision
## Every frame is then the sensor dataframe joined to this rolling max calculation.
event_id = 1128590
event_df = train.loc[train["event_id"] == event_id]
max_frame = event_df.shape[0]
frame = 20
sensor_fig = remove_fig_info(sensor_fig)


def get_frame(frame_number, event_df, sensor_fig):
    """
    Creates a plotly 3dScatter plot after frame_number of interactions
    """

    frame_df = event_df.iloc[0:frame]
    frame_df = frame_df.groupby(["sensor_id"], as_index=False).agg({"charge": "max"})

    frame_geometry = sensor_geometry.merge(
        frame_df,
        how="inner",
        on="sensor_id",
    )

    frame_fig = px.scatter_3d(
        frame_geometry, x="x", y="y", z="z", opacity=0.7, size="charge", color="charge"
    )

    fig = go.Figure(
        data=[sensor_fig.data[0], frame_fig.data[0]],
        layout=dict(
            hovermode=False,
            paper_bgcolor='black',
            scene=dict(
                xaxis=dict(showticklabels=False),
                yaxis=dict(showticklabels=False),
                zaxis=dict(showticklabels=False),
                xaxis_visible=False,
                yaxis_visible=False,
                zaxis_visible=False,
            ),
        ),
    )
    return fig

fig = get_frame(1000, event_df, sensor_fig)
fig.show()

In [103]:
train.groupby('event_id').agg({'time' : 'count'}).sort_values(by='time', ascending=True)

,time
event_id,
303580,15
3201121,15
1317057,16
1471871,17
107144,17
...,...
2426177,98405
2097125,98594
759693,108737
